In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [2]:
dataset = 'emma' # onion or emma
results_dir = f'../results/' + dataset + '/performance/'
cutoffs = [1, 5, 10, 20]
metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR', 'EFD', 'EPC', 'ARP', 'PopREO', 'PopRSP', 'ItemCoverage', 'UserCoverage']
#metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR']

In [3]:
def find_modality(string):
    keyword = "modalites="
    start = string.find(keyword)
    if start == -1:
        return None

    start += len(keyword)
    end = string.find("_", start)
    if end == -1:
        end = len(string)

    modality = string[start:end].strip("'")
    return modality

In [19]:
# read all files in result folder
files = os.listdir(results_dir)
df = pd.DataFrame()

for cutoff in cutoffs:
    df_cut = pd.DataFrame()
    result_files_cutoff = [f for f in files if f.startswith(f'rec_cutoff_{cutoff}_')]
    for f in result_files_cutoff:
        df_tmp = pd.read_csv(results_dir + f, sep='\t')
        tmp_metrics = [m for m in metrics if m in df_tmp.columns]
        df_tmp.rename(columns={m: f'{m}@{cutoff}' for m in tmp_metrics}, inplace=True)
        df_tmp['modality'] = find_modality(df_tmp['model'].values[0])
        df_tmp['model'] = df_tmp['model'].str.split('_').str[0]
        df_tmp['date'] = f[-23:-4]
        df_cut = pd.concat([df_cut, df_tmp], axis=0)

    # keep only newest model results
    df_cut = df_cut.sort_values('date').drop_duplicates(subset=['model', 'modality'], keep='last')

    # merge with previous cutoffs
    if not df.empty:
        df = pd.merge(df, df_cut, on=['model', 'date', 'modality'], how='inner')
        print(df.shape)
    else:
        df = df_cut

df = df[['model', 'modality', 'date'] + [c for c in df.columns if '@' in c]]
df.sort_values('nDCG@5', inplace=True)
# display columns with @10 only
#df[['model', 'modality', 'date'] + [c for c in df.columns if '@1' in c]]
df[['model', 'modality', 'date'] + ['Recall@1', 'Recall@5', 'Recall@10', 'nDCG@1', 'nDCG@5', 'nDCG@10']]

(7, 27)
(7, 39)
(7, 51)


,model,modality,date,Recall@1,Recall@5,Recall@10,nDCG@1,nDCG@5,nDCG@10
0,AttributeItemKNN,None,2024_12_18_09_58_00,0.026316,0.031579,0.050000,0.026316,0.029769,0.037963
2,FeatureItemKNN,None,2025_01_14_09_55_06,0.026316,0.049123,0.054386,0.026316,0.040170,0.042614
1,BiVAECF,None,2024_12_18_10_55_07,0.034461,0.034461,0.052882,0.078947,0.044166,0.048992
6,FeatureItemKNN,emotion,2025_01_14_13_59_39,0.141379,0.357615,0.486638,0.224138,0.285668,0.333358
5,FeatureItemKNN,textual,2025_01_14_13_59_19,0.139943,0.366810,0.504598,0.221264,0.289479,0.339419
4,ItemKNN,None,2025_01_14_13_57_38,0.141379,0.368151,0.493822,0.224138,0.291800,0.337357
3,FeatureItemKNN,audio,2025_01_14_13_55_13,0.152874,0.356657,0.478017,0.238506,0.292085,0.335379


In [9]:
# export to comma seperated file
df.to_csv(f'../results/{dataset}_performance.csv', index=False)

In [31]:
for cutoff in cutoffs:
    df_cut = df[['model'] + [c for c in df.columns if f'@{cutoff}' in c]]
    df_cut.to_csv(f'../results/{dataset}_performance_{cutoff}.csv', index=False)